In [1]:
import json
import pandas as pd

import aidd.sys.config as cfg
from aidd.sys.data_io import get_merged_data, read_data

In [2]:
mdict = get_merged_data()

[ee25d04a7ed4][2024-03-23 22:28:08.912947] 1차 전처리 완료된 데이터 불러오기 시작
[ee25d04a7ed4][2024-03-23 22:28:08.939281]   1차 전처리된 공사비 데이터 셋: 크기(15335, 8), 처리시간(0:00:00.026276)
[ee25d04a7ed4][2024-03-23 22:28:08.975172]   1차 전처리된 전주 데이터 셋: 크기(27896, 6), 처리시간(0:00:00.035839)
[ee25d04a7ed4][2024-03-23 22:28:09.053290]   1차 전처리된 전선 데이터 셋: 크기(30348, 11), 처리시간(0:00:00.078051)
[ee25d04a7ed4][2024-03-23 22:28:09.066535]   1차 전처리된 인입선 데이터 셋: 크기(17034, 6), 처리시간(0:00:00.013195)
[ee25d04a7ed4][2024-03-23 22:28:09.066569] 1차 전처리 완료된 데이터 불러오기 종료, 최종 처리시간: 0:00:00.153629


In [50]:
def make_json(cons_id=None, mdict=None):
    sdict = get_sample_data(cons_id, mdict)
    jdict = {}
    for _, row in sdict["CONS"].iterrows():
        jdict["BASIC"] = {}
        jdict["BASIC"].update({
            "ACC_NO": row["CONS_ID"],
            # 나중에 이부분이 날짜형이면 아래처럼 변경
            "ACC_DATE": row["LAST_MOD_DATE"],
            # "ACC_DATE": row["LAST_MOD_DATE"].dt.strftime("%Y-%m-%d %H:%M:%S"),
            "OFFICE_NAME": row["OFFICE_NAME"],
            "CONT_CAP": row["CONT_CAP"],
            "ACC_TYPE_NAME": row["ACC_TYPE_NAME"]
        })
        
        jdict["PREDICT"] = {
            "PRED_1":{
                "BASIC":{
                    "PRED_NO": 1,
                    "PRED_TYPE": 1,
                }, 
                "POLE":{}, "LINE":{}, "SL": {}
            }
        }
        df = sdict["POLE"]
        df[["X", "Y", "TEMP1", "TEMP2"]] = df.COORDINATE.str.split(",", expand=True)
        idx = 1
        for _, row in df.iterrows():
            jdict["PREDICT"]["PRED_1"]["POLE"][f"POLE_{idx}"] = {}
            jdict["PREDICT"]["PRED_1"]["POLE"][f"POLE_{idx}"].update({
                "POLE_SEQ": idx,
                "GEO_X": row["X"],
                "GEO_Y": row["Y"],
                "POLE_SHAPE_CD": row["POLE_SHAPE_CD"],
                "POLE_TYPE_CD": row["POLE_TYPE_CD"],
                "POLE_SPEC_CD": row["POLE_SPEC_CD"],
            })
            idx += 1
        jdict["PREDICT"]["PRED_1"]["BASIC"]["POLE_CNT"] = idx-1
        
        df = sdict["LINE"]
        idx = 1
        for _, row in df.iterrows():
            jdict["PREDICT"]["PRED_1"]["LINE"][f"LINE_{idx}"] = {}
            jdict["PREDICT"]["PRED_1"]["LINE"][f"LINE_{idx}"].update({
                "LINE_SEQ": idx,
                "WIRING_SCHEME": row["WIRING_SCHEME"],
                "SPAN": row["SPAN"],
                "LINE_TYPE_CD": row["LINE_TYPE_CD"],
                "LINE_SPEC_CD": row["LINE_SPEC_CD"],
                "LINE_PHASE_CD": row["LINE_PHASE_CD"],
                "NEUTRAL_TYPE_CD": row["NEUTRAL_TYPE_CD"],
                "NEUTRAL_SPEC_CD": row["NEUTRAL_SPEC_CD"],
            })
            idx += 1
        jdict["PREDICT"]["PRED_1"]["BASIC"]["LINE_CNT"] = idx-1
        
        df = sdict["SL"]
        idx = 1
        for _, row in df.iterrows():
            jdict["PREDICT"]["PRED_1"]["SL"][f"SL_{idx}"] = {}
            jdict["PREDICT"]["PRED_1"]["SL"][f"SL_{idx}"].update({
                "SL_SEQ": idx,
                "SL_TYPE_CD": row["SL_TYPE_CD"],
                "SL_SPEC_CD": row["SL_SPEC_CD"],
                "SPAN": row["SPAN"],
                "SUPERVISOR": row["SUPERVISOR"],
            })
            idx += 1
        jdict["PREDICT"]["PRED_1"]["BASIC"]["SL_CNT"] = idx-1
    return jdict
    
def get_sample_data(cons_id, mdict):
    sdict = {}
    for key in mdict.keys():
        _df = mdict[key]
        _df = _df[_df.CONS_ID==cons_id].copy()
        sdict[key] = _df
    return sdict

In [51]:
CONS_ID = "477420204194"
jdata = make_json(cons_id=CONS_ID, mdict=mdict)
jdata = json.dumps(jdata, ensure_ascii=False)
jdata

'{"BASIC": {"ACC_NO": "477420204194", "ACC_DATE": "2020-12-23 11:50:39", "OFFICE_NAME": "단양지사", "CONT_CAP": 3, "ACC_TYPE_NAME": "신설(상용/임시)"}, "PREDICT": {"PRED_1": {"BASIC": {"PRED_NO": 1, "PRED_TYPE": 1, "POLE_CNT": 4, "LINE_CNT": 4, "SL_CNT": 1}, "POLE": {"POLE_1": {"POLE_SEQ": 1, "GEO_X": "128.367723904178", "GEO_Y": "37.0838155358327", "POLE_SHAPE_CD": "O", "POLE_TYPE_CD": "C", "POLE_SPEC_CD": 12.0}, "POLE_2": {"POLE_SEQ": 2, "GEO_X": "128.367812235705", "GEO_Y": "37.0842970919574", "POLE_SHAPE_CD": "O", "POLE_TYPE_CD": "H", "POLE_SPEC_CD": 14.0}, "POLE_3": {"POLE_SEQ": 3, "GEO_X": "128.368262155256", "GEO_Y": "37.0845003923504", "POLE_SHAPE_CD": "O", "POLE_TYPE_CD": "H", "POLE_SPEC_CD": 14.0}, "POLE_4": {"POLE_SEQ": 4, "GEO_X": "128.368735480787", "GEO_Y": "37.0846605693381", "POLE_SHAPE_CD": "O", "POLE_TYPE_CD": "H", "POLE_SPEC_CD": 14.0}}, "LINE": {"LINE_1": {"LINE_SEQ": 1, "WIRING_SCHEME": 13, "SPAN": 54, "LINE_TYPE_CD": "AO", "LINE_SPEC_CD": 35.0, "LINE_PHASE_CD": 1, "NEUTRAL_

In [52]:
def json_to_dataframe(data):
    data = json.loads(data)
    jdata = {}
    df_dict = {}
    basic = data["BASIC"]
    for key, value in data["PREDICT"].items():
        df_dict[key] = {}
        jdata[key] = value
        jdata[key]["BASIC"]["ACC_NO"] = basic["ACC_NO"]
        _j = jdata[key]['BASIC']
        _j['CONS_ID'] = _j.pop('ACC_NO')
        _j['LAST_MOD_DATE'] = basic['ACC_DATE']
        _j['OFFICE_NAME'] = basic['OFFICE_NAME']
        _j['CONT_CAP'] = basic['CONT_CAP']
        _j['ACC_TYPE_NAME'] = basic['ACC_TYPE_NAME']
        _j['TOTAL_CONS_COST'] = 0
        df_dict[key]['CONS'] = pd.DataFrame([_j])
        
        _p = {
            'CONS_ID': [], 'POLE_SHAPE_CD': [], 
            'POLE_TYPE_CD': [], 'POLE_SPEC_CD': [],
            'COORDINATE': []
        }
        for _k, _v in value['POLE'].items():
            _p['CONS_ID'].append(basic['ACC_NO'])
            _p['POLE_SHAPE_CD'].append(_v['POLE_SHAPE_CD'])
            _p['POLE_TYPE_CD'].append(_v['POLE_TYPE_CD'])
            _p['POLE_SPEC_CD'].append(_v['POLE_SPEC_CD'])
            _p['COORDINATE'].append(f'{_v["GEO_X"]},{_v["GEO_Y"]},1,1')
        df_dict[key]['POLE'] = pd.DataFrame(_p)
        
        _l = {
            'CONS_ID': [], 'WIRING_SCHEME': [], 
            'LINE_TYPE_CD': [], 'LINE_SPEC_CD': [],
            'LINE_PHASE_CD': [], 'SPAN': [], 
            'NEUTRAL_TYPE_CD': [], 'NEUTRAL_SPEC_CD': [],
        }
        for _k, _v in value['LINE'].items():
            _l['CONS_ID'].append(basic['ACC_NO'])
            _l['WIRING_SCHEME'].append(_v['WIRING_SCHEME'])
            _l['LINE_TYPE_CD'].append(_v['LINE_TYPE_CD'])
            _l['LINE_SPEC_CD'].append(_v['LINE_SPEC_CD'])
            _l['LINE_PHASE_CD'].append(_v['LINE_PHASE_CD'])
            _l['SPAN'].append(_v['SPAN'])
            _l['NEUTRAL_TYPE_CD'].append(_v['NEUTRAL_TYPE_CD'])
            _l['NEUTRAL_SPEC_CD'].append(_v['NEUTRAL_SPEC_CD'])
        df_dict[key]['LINE'] = pd.DataFrame(_l)
        
        _s = {
            'CONS_ID': [], 'SL_TYPE_CD': [], 
            'SL_SPEC_CD': [], 'SPAN': [],
            'SUPERVISOR': []
        }
        for _k, _v in value['SL'].items():
            _s['CONS_ID'].append(basic['ACC_NO'])
            _s['SL_TYPE_CD'].append(_v['SL_TYPE_CD'])
            _s['SL_SPEC_CD'].append(_v['SL_SPEC_CD'])
            _s['SUPERVISOR'].append(_v['SUPERVISOR'])
            _s['SPAN'].append(_v['SPAN'])
        df_dict[key]['SL'] = pd.DataFrame(_s)
            
    return jdata, df_dict

In [53]:
rjdata, df_dict = json_to_dataframe(jdata)

In [54]:
df_dict['PRED_1']['CONS'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRED_NO          1 non-null      int64 
 1   PRED_TYPE        1 non-null      int64 
 2   POLE_CNT         1 non-null      int64 
 3   LINE_CNT         1 non-null      int64 
 4   SL_CNT           1 non-null      int64 
 5   CONS_ID          1 non-null      object
 6   LAST_MOD_DATE    1 non-null      object
 7   OFFICE_NAME      1 non-null      object
 8   CONT_CAP         1 non-null      int64 
 9   ACC_TYPE_NAME    1 non-null      object
 10  TOTAL_CONS_COST  1 non-null      int64 
dtypes: int64(7), object(4)
memory usage: 216.0+ bytes


In [55]:
df_dict['PRED_1']['POLE']

,CONS_ID,POLE_SHAPE_CD,POLE_TYPE_CD,POLE_SPEC_CD,COORDINATE
0,477420204194,O,C,12.0,"128.367723904178,37.0838155358327,1,1"
1,477420204194,O,H,14.0,"128.367812235705,37.0842970919574,1,1"
2,477420204194,O,H,14.0,"128.368262155256,37.0845003923504,1,1"
3,477420204194,O,H,14.0,"128.368735480787,37.0846605693381,1,1"


In [56]:
df_dict['PRED_1']['LINE']

,CONS_ID,WIRING_SCHEME,LINE_TYPE_CD,LINE_SPEC_CD,LINE_PHASE_CD,SPAN,NEUTRAL_TYPE_CD,NEUTRAL_SPEC_CD
0,477420204194,13,AO,35.0,1,54,AL,32.0
1,477420204194,13,AO,35.0,1,53,AL,58.0
2,477420204194,13,AO,35.0,1,46,AL,58.0
3,477420204194,13,AO,35.0,1,49,AL,58.0


In [57]:
df_dict['PRED_1']['SL']

,CONS_ID,SL_TYPE_CD,SL_SPEC_CD,SPAN,SUPERVISOR
0,477420204194,D2,3.2,7,1
